In [1]:
import os

os.chdir("..")

In [2]:
from datasetsforecast.m3 import M3
from utilsforecast.plotting import plot_series
import pandas as pd
from models.NBeatsStackMoe import NBeatsStackMoe
from models.NBeatsMoeLags import NBeatsMoeLags
from neuralforecast import NeuralForecast
from utils import load_dataset, train_test_split
from models.callbacks.probs_collector import GateValuesCollectorCallback
from neuralforecast.losses.numpy import smape
from neuralforecast.models import NBEATS
from neuralforecast.tsdataset import TimeSeriesDataset
import matplotlib.pyplot as plt
from datasets.load_data.gluonts_dataset import GluontsDataset
from models.hyper import config
import numpy as np
import random
from neuralforecast.losses.pytorch import HuberLoss, MAE, MSE, SMAPE, MAPE, MASE
from functools import partial

# Load the M3 dataset
m3 = M3.load(directory='C:\\Users\\ricar\\mixture_of_experts_time_series\\data\\m3\\', group='Monthly')[0]


c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-01 13:26:32,765	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-06-01 13:26:33,003	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
c:\Users\ricar\mixture_of_experts_time_series\venv\Lib\site-packages\gluonts\json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
horizon = 18

In [4]:
_random_conf = {
    "loss": ['SMAPE', 'HuberLoss', 'MAPE'],
    "input_size": [int(horizon * x) for x in [1.5, 2, 2.5, 3, 4, 5]],
}

MASE = partial(MASE, horizon=1)

def get_random_conf(conf = None):
    if conf is None:
        conf = config['m3']['Monthly']['NBeatsStackMoe']
    conf["loss"] = eval(random.choice(_random_conf["loss"]))()
    conf["input_size"] = random.choice(_random_conf["input_size"])
    conf["random_seed"] = random.randint(0, 10000)

    # conf["start_padding_enabled"] = True
    return conf


In [5]:
def calculate_smape(Y_test_df, Y_hat_df, forecast_col):
    """Calculate the sMAPE."""
    y_true = Y_test_df['y'].values
    try:
        y_hat = Y_hat_df[forecast_col].values
    except Exception as exc:
        y_hat = Y_hat_df
   
    n_series = Y_test_df['unique_id'].nunique()
    try:
        y_true = y_true.reshape(n_series, -1)
        y_hat = y_hat.reshape(n_series, -1)
    except Exception as e:
        raise ValueError("Error reshaping arrays") from e

    return smape(y_true, y_hat)

In [6]:
y_train, y_test = train_test_split(m3, horizon)

models = []

for i in range(180):
    random_conf = get_random_conf()
    model = NBeatsStackMoe(
        h=horizon,
        store_all_gate_logits=False,
        **random_conf,
    )
    models.append(model)

nf = NeuralForecast(
    models=models,
    freq='M',
)

nf.fit(df=y_train, static_df=None, val_size=horizon)
predictions = nf.predict(futr_df=y_test)


nbeats_cols = [col for col in predictions.columns if "NBeatsStack" in col]
predictions["prediction_agg"] = predictions[nbeats_cols].median(axis=1)


smape_score = calculate_smape(y_test, predictions, "prediction_agg")
print(f"sMAPE score: {smape_score:.4f}")


INFO:lightning_fabric.utilities.seed:Seed set to 8709
INFO:lightning_fabric.utilities.seed:Seed set to 5886
INFO:lightning_fabric.utilities.seed:Seed set to 719
INFO:lightning_fabric.utilities.seed:Seed set to 1831
INFO:lightning_fabric.utilities.seed:Seed set to 9951
INFO:lightning_fabric.utilities.seed:Seed set to 1554
INFO:lightning_fabric.utilities.seed:Seed set to 1394
INFO:lightning_fabric.utilities.seed:Seed set to 7356
INFO:lightning_fabric.utilities.seed:Seed set to 7213
INFO:lightning_fabric.utilities.seed:Seed set to 8191
INFO:lightning_fabric.utilities.seed:Seed set to 9746
INFO:lightning_fabric.utilities.seed:Seed set to 9931
INFO:lightning_fabric.utilities.seed:Seed set to 6495
INFO:lightning_fabric.utilities.seed:Seed set to 2433
INFO:lightning_fabric.utilities.seed:Seed set to 6866
INFO:lightning_fabric.utilities.seed:Seed set to 1418
INFO:lightning_fabric.utilities.seed:Seed set to 7709
INFO:lightning_fabric.utilities.seed:Seed set to 6817
INFO:lightning_fabric.utiliti

Epoch 166:  67%|██████▋   | 4/6 [00:00<00:00,  8.41it/s, v_num=708, train_loss_step=0.115, train_loss_epoch=0.135, valid_loss=0.156]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 166:  67%|██████▋   | 4/6 [00:00<00:00,  8.41it/s, v_num=708, train_loss_step=0.115, train_loss_epoch=0.135, valid_loss=0.156]


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAPE          | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 655 K  | train
4 | gate         | Sequential    | 1.1 K  | train
-------------------------------------------------------
654 K     Trainable params
2.0 K     Non-trainable params
656 K     Total params
2.626     Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode


...

INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 34.40it/s]

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True


INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 42.67it/s]
sMAPE score: 0.1358


In [ ]:
print(f"sMAPE score: {smape_score:.4f}")

sMAPE score: 0.1358


,unique_id,ds,NBeatsStackMoe,NBeatsStackMoe1,NBeatsStackMoe2,NBeatsStackMoe3,NBeatsStackMoe4,NBeatsStackMoe5,NBeatsStackMoe6,NBeatsStackMoe7,...,NBeatsStackMoe171,NBeatsStackMoe172,NBeatsStackMoe173,NBeatsStackMoe174,NBeatsStackMoe175,NBeatsStackMoe176,NBeatsStackMoe177,NBeatsStackMoe178,NBeatsStackMoe179,prediction_agg
0,M1,1994-03-31,2101.528564,1825.395142,2286.539307,2025.922241,1269.381470,1757.979126,2341.724121,2125.269287,...,1935.692383,1594.380615,2357.999756,2071.433105,1871.214966,2636.430176,2326.763184,1978.666992,998.827271,2025.791992
1,M1,1994-04-30,2274.670898,2063.103760,2446.326416,2095.622070,1783.833496,2303.599121,2452.346191,2298.683594,...,2353.343506,1893.711182,2757.432617,2261.038818,2333.444092,2837.181641,2541.112305,2212.013428,2036.060425,2301.141357
2,M1,1994-05-31,2102.538086,2067.278809,2274.228516,1997.196777,1475.619385,2021.120605,1852.051636,1814.972412,...,1749.686157,1758.091919,2587.179199,2060.080811,2016.134888,2297.133789,2342.561035,1893.704346,2098.235596,2060.750977
3,M1,1994-06-30,2045.495483,2024.559570,2196.927246,1794.354736,1432.061035,1951.099609,1986.947021,2037.842773,...,1894.915283,1687.344849,2723.471191,2191.814697,2287.014160,2339.958984,2183.470703,2094.336670,1941.315186,2043.980225
4,M1,1994-07-31,1965.618530,1961.222534,2705.034424,2312.720215,1479.777710,2301.998779,1990.230347,2469.373535,...,2211.367676,2178.807129,2737.469971,2295.766602,2257.408936,2075.398926,2650.653320,2222.392822,2305.145264,2251.844238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25699,M999,1993-10-31,5452.085938,5381.921387,5290.620117,5299.441406,5211.586914,5187.951172,5436.750000,5355.261230,...,5357.542969,5344.971680,5365.932617,5276.649902,5458.384277,5485.014648,5330.835449,5370.976074,5227.456543,5367.019043
25700,M999,1993-11-30,5462.571777,5387.220703,5325.860352,5286.467773,5189.045898,5181.220703,5470.052734,5369.250488,...,5360.397461,5358.431641,5375.031250,5293.872559,5470.813965,5487.279297,5343.456055,5374.973633,5240.738770,5375.002441
25701,M999,1993-12-31,5497.972168,5405.514648,5339.976074,5288.808594,5170.925293,5173.499512,5461.801758,5383.061523,...,5365.534180,5358.430176,5390.463867,5283.331055,5478.517578,5492.114746,5353.147461,5398.163086,5279.960449,5386.859375
25702,M999,1994-01-31,5505.111816,5427.038086,5332.618652,5300.666016,5176.295898,5166.148926,5467.588867,5406.618164,...,5352.950195,5378.796387,5399.471680,5297.230957,5495.530762,5511.156250,5375.994141,5417.699707,5264.270020,5395.060059


In [ ]:

predictions.save("predictions_m3_monthly_nbeats_stack_moe.csv", index=False)

In [89]:
y_train, y_test = train_test_split(m3, horizon)

models = []

for i in range(18):
    random_conf = get_random_conf(config['m3']['Monthly']['NBEATS'])
    model = NBEATS(
        h=horizon,
        **random_conf,
    )
    models.append(model)

nf = NeuralForecast(
    models=models,
    freq='M',
)

nf.fit(df=y_train, static_df=None, val_size=horizon)
predictions = nf.predict(futr_df=y_test)


nbeats_cols = [col for col in predictions.columns if "NBEATS" in col]
predictions["prediction_agg"] = predictions[nbeats_cols].median(axis=1)


smape_score = calculate_smape(y_test, predictions, "prediction_agg")
print(f"sMAPE score: {smape_score:.4f}")


INFO:lightning_fabric.utilities.seed:Seed set to 5288
INFO:lightning_fabric.utilities.seed:Seed set to 5124
INFO:lightning_fabric.utilities.seed:Seed set to 7101
INFO:lightning_fabric.utilities.seed:Seed set to 9229
INFO:lightning_fabric.utilities.seed:Seed set to 7657
INFO:lightning_fabric.utilities.seed:Seed set to 5585
INFO:lightning_fabric.utilities.seed:Seed set to 9802
INFO:lightning_fabric.utilities.seed:Seed set to 276
INFO:lightning_fabric.utilities.seed:Seed set to 3248
INFO:lightning_fabric.utilities.seed:Seed set to 1640
INFO:lightning_fabric.utilities.seed:Seed set to 8224
INFO:lightning_fabric.utilities.seed:Seed set to 8464
INFO:lightning_fabric.utilities.seed:Seed set to 6697
INFO:lightning_fabric.utilities.seed:Seed set to 9621
INFO:lightning_fabric.utilities.seed:Seed set to 8617
INFO:lightning_fabric.utilities.seed:Seed set to 6553
INFO:lightning_fabric.utilities.seed:Seed set to 2728
INFO:lightning_fabric.utilities.seed:Seed set to 5138
INFO:pytorch_lightning.utilit

Epoch 208:  33%|███▎      | 4/12 [00:00<00:01,  5.14it/s, v_num=569, train_loss_step=0.138, train_loss_epoch=0.123, valid_loss=0.158]  

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=2500` reached.


Epoch 208:  33%|███▎      | 4/12 [00:00<00:01,  5.11it/s, v_num=569, train_loss_step=0.138, train_loss_epoch=0.123, valid_loss=0.158]


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAPE          | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 211 K  | train
-------------------------------------------------------
207 K     Trainable params
4.0 K     Non-trainable params
211 K     Total params
0.844     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


...

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 43.65it/s]

INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 12/12 [00:00<00:00, 42.09it/s]
sMAPE score: 0.1356


In [87]:
predictions

,unique_id,ds,NBeatsStackMoe,NBeatsStackMoe1,NBeatsStackMoe2,NBeatsStackMoe3,NBeatsStackMoe4,NBeatsStackMoe5,NBeatsStackMoe6,NBeatsStackMoe7,...,NBeatsStackMoe9,NBeatsStackMoe10,NBeatsStackMoe11,NBeatsStackMoe12,NBeatsStackMoe13,NBeatsStackMoe14,NBeatsStackMoe15,NBeatsStackMoe16,NBeatsStackMoe17,prediction_agg
0,M1,1994-03-31,2027.350220,1976.977783,2159.812500,2212.591064,2662.133545,1650.762939,1426.460205,2394.214355,...,2100.765381,2075.814941,2048.272461,1254.936646,2010.218994,2305.779785,2076.729004,2272.338135,2319.632080,NaN
1,M1,1994-04-30,2233.364990,2896.877686,2556.416260,2702.814453,3003.040771,1697.746094,1848.511841,2649.683594,...,2327.843262,2069.480957,2485.066162,1524.755249,2157.153809,2439.037842,2418.768066,2424.003418,2503.113037,NaN
2,M1,1994-05-31,2034.908936,2397.562744,2302.072266,2640.079834,2491.428711,1721.888794,1805.814575,2363.928467,...,1989.861206,2059.361816,2121.086426,1306.571411,1935.060425,2396.496094,1903.889160,2241.905762,2264.243896,NaN
3,M1,1994-06-30,1957.655029,2563.429443,2246.309814,2723.009277,2422.212891,1501.725342,1692.576904,2116.087891,...,1933.465942,2190.037842,1974.409668,1224.810303,1952.374512,2266.936279,1859.706055,2106.559082,2543.100098,NaN
4,M1,1994-07-31,2608.313965,2538.428955,2602.689209,3166.058350,3275.409912,1834.110474,2113.342773,2077.098633,...,2366.277832,2123.212891,1916.170532,1449.702881,2340.781006,2468.893555,2079.978516,1965.350342,2710.783447,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25699,M999,1993-10-31,5377.535645,5299.405273,5358.665527,5356.336426,5273.133301,5407.424805,5227.890625,5506.175781,...,5387.047363,5437.611328,5306.417480,5245.458984,5283.004883,5357.413086,5435.615723,5438.089355,5346.955078,NaN
25700,M999,1993-11-30,5384.088379,5283.923340,5358.282227,5360.583496,5285.469238,5432.648926,5201.810059,5525.135254,...,5395.036621,5444.592773,5323.011230,5247.215820,5291.891602,5359.339844,5447.455566,5430.526367,5351.978027,NaN
25701,M999,1993-12-31,5401.246582,5321.370605,5375.719727,5356.472168,5311.429199,5428.625488,5191.773926,5549.574707,...,5405.485840,5429.930664,5305.908691,5272.377930,5299.737305,5374.971680,5468.077637,5445.824707,5364.867676,NaN
25702,M999,1994-01-31,5414.285156,5311.018555,5389.261719,5345.674805,5253.677246,5427.290527,5177.031738,5546.982422,...,5409.825195,5447.159668,5310.714355,5272.042480,5302.655762,5378.166504,5473.582031,5485.347656,5368.282715,NaN
